Kiểm tra và xử lý lại sắc thái (sentiment) của từng bài viết / bình luận cho chính xác

In [1]:
import pandas as pd
from textblob import TextBlob
from deep_translator import GoogleTranslator

# Hàm dịch tiếng Việt sang tiếng Anh và phân tích sắc thái
def classify_sentiment_vi(text):
    try:
        # Dịch sang tiếng Anh
        translated = GoogleTranslator(source='vi', target='en').translate(str(text))
        analysis = TextBlob(translated)
        polarity = analysis.sentiment.polarity
        if polarity > 0:
            return "positive"
        elif polarity < 0:
            return "negative"
        else:
            return "neutral"
    except:
        return "neutral"

# Đọc file Excel
file_path = 'Social Media Data.xlsx'
df = pd.read_excel(file_path, sheet_name='Dữ liệu')

# Lọc các dòng có content hợp lệ
df_valid = df[df['Content'].notna()].copy()

# Chuẩn hóa sentiment gốc
df_valid['Sentiment'] = df_valid['Sentiment'].astype(str).str.strip().str.lower()

# Phân tích sentiment mới bằng nội dung đã dịch
df_valid['sentiment_rechecked'] = df_valid['Content'].apply(classify_sentiment_vi)

# Chuẩn hóa sentiment_rechecked để tránh lỗi so sánh
df_valid['sentiment_rechecked'] = df_valid['sentiment_rechecked'].astype(str).str.strip().str.lower()

# So sánh xem giống hay khác
df_valid['so_sanh'] = df_valid.apply(
    lambda x: 'giống' if x['Sentiment'] == x['sentiment_rechecked'] else 'khác',
    axis=1
)

# Lấy các cột cần thiết
df_Sentiment = df_valid[['Platform', 'Type', 'Title', 'Content', 'Sentiment', 'sentiment_rechecked', 'so_sanh']]
df_Sentiment.columns = [col.lower() for col in df_Sentiment.columns]

# Ghi ra file Excel, thêm vào sheet 'Sentiment'
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_Sentiment.to_excel(writer, sheet_name='Sentiment', index=False)

Nội dung mỗi bài viết và bình luận đang thể hiện khía cạnh gì

In [2]:
import pandas as pd

# Đọc dữ liệu từ file Excel
file_path = 'Social Media Data.xlsx'
df = pd.read_excel(file_path, sheet_name='Dữ liệu')

# Hàm phân loại khía cạnh theo từ khóa
def classify_aspect(text):
    text = str(text).lower()
    
    # Từ khóa đặc trưng
    keywords_brand = ['tcbs', 'thương hiệu', 'chứng khoán kỹ thuật số', 'giới thiệu tcbs']
    keywords_product = ['trái phiếu', 'quỹ đầu tư', 'ứng dụng', 'sản phẩm', 'dịch vụ', 'ibond', 'tcinvest', 'cổ phiếu']
    keywords_promo = ['khuyến mãi', 'ưu đãi', 'giảm phí', 'voucher', 'tặng', 'quà', 'miễn phí']
    
    # Phân loại
    if any(keyword in text for keyword in keywords_brand):
        return 'Thương hiệu'
    elif any(keyword in text for keyword in keywords_product):
        return 'Sản phẩm'
    elif any(keyword in text for keyword in keywords_promo):
        return 'Chương trình khuyến mãi'
    else:
        return 'Khác'

# Áp dụng hàm phân loại vào cột Content
df['aspect'] = df['Content'].fillna('').apply(classify_aspect)

# Chọn cột xuất ra và đổi tên thành chữ thường
df_output = df[['Platform', 'Type', 'Title', 'Content', 'aspect']].copy()
df_output.columns = ['platform', 'type', 'title', 'content', 'aspect']

# Ghi vào sheet mới trong file Excel gốc
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    df_output.to_excel(writer, sheet_name='Aspect', index=False)